In [ ]:
# Import paths
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import hstack
from sklearn.metrics import ndcg_score

import joblib

DATA = OUT = Path.cwd()
OUT.mkdir(parents=True, exist_ok=True)
print("DATA =", DATA, "OUT =", OUT)

train = pd.read_csv(DATA / "train_program.csv")
test  = pd.read_csv(DATA / "test_program.csv")

print(f"Dataset sizes: train={len(train)}, test={len(test)}")


In [ ]:
# Feature engineering: MLB one-hot on interests and field_tags
def to_list(s: str):
    return [x.strip().lower() for x in str(s).split(";") if x.strip()]

mlb_int = MultiLabelBinarizer(sparse_output=True)
mlb_tag = MultiLabelBinarizer(sparse_output=True)

X_tr_sparse = hstack([
    mlb_int.fit_transform(train["interests"].map(to_list)),
    mlb_tag.fit_transform(train["field_tags"].map(to_list))
], format="csr")

X_te_sparse = hstack([
    mlb_int.transform(test["interests"].map(to_list)),
    mlb_tag.transform(test["field_tags"].map(to_list))
], format="csr")

y_train = train["label_match"].values
y_test  = test["label_match"].values

X_tr_sparse.shape, X_te_sparse.shape


In [ ]:
# Choose KNN variant
knn = KNeighborsRegressor(n_neighbors=5, weights="distance", metric="cosine", n_jobs=-1)

In [ ]:
# Train
knn.fit(X_tr_sparse, y_train)
test_pred = knn.predict(X_te_sparse)


In [ ]:
# Save artifacts
def save_scored(df_split, preds, fname):
    out = df_split.copy()
    out["pred_label_match"] = np.round(preds, 4)
    out.to_csv(OUT / fname, index=False)

save_scored(test, test_pred, "knn_program_test.csv")

In [ ]:

val_df = test[["student_id", "label_match"]].copy()
val_df["pred_label_match"] = np.asarray(test_pred, dtype=float)

rows = []
for sid, g in val_df.groupby("student_id", sort=True):
    y_true = g["label_match"].to_numpy().reshape(1, -1)
    y_pred = g["pred_label_match"].to_numpy().reshape(1, -1)
    rows.append({"student_id": sid, "nDCG@3": float(ndcg_score(y_true, y_pred, k=3))})

ndcg_KNN = pd.DataFrame(rows).sort_values("student_id").reset_index(drop=True)
mean_ndcg_KNN = float(ndcg_KNN["nDCG@3"].mean())

display(ndcg_KNN)
print("Mean nDCG@3", round(mean_ndcg_KNN, 6))
